## Sentiment Analysis

### Sentiment Analysis using NLTK VADER Package along with Threshold Optimization

In [ ]:
# We will use this data- https://archive.ics.uci.edu/m1/datasets/sentiment+labelled+sentences
# Save this data in a csv and import in Python
# Please review the 'readme' in the downloaded zip folder to get to know more about this data
import numpy as np
import pandas as pd
import nltk
nltk.download ('all')
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Magic command to print multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
df = pd.read_csv('yelp_sentiment (1).csv', encoding = 'unicode_escape')
df.head()
df.shape
df.columns
#Install and upgrade VADER
!pip install vaderSentiment
!pip install --upgrade vaderSentiment

,Text,Sentiment
0,!....THE OWNERS REALLY REALLY need to quit bei...,0
1,"(It wasn't busy either) Also, the building was...",0
2,(The bathroom is just next door and very nice.),1
3,* Both the Hot & Sour & the Egg Flower Soups w...,1
4,2 Thumbs Up!!,1


(996, 2)

Index(['Text', 'Sentiment'], dtype='object')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.1 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgrade vaderSentiment


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean_text(text):
  if isinstance(text, str):
        # Convert the text to lowercase
        text = text.lower()
        # Remove special characters and digits
        text = re.sub(r'[^a-z\s]', '', text)
        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', '', text)
        # Remove Punctuations
        text = re.sub(r'[^\w\s]', '', text)
        # Remove Numbers
        text = re.sub(r'\d+', '', text)
        # Remove extra white spaces
        text = re.sub(r'\s+', ' ', text).strip()
        # Remove @ mentions
        text = re.sub(r'@\w+', '', text)
        # Remove hashtags
        text = re.sub(r'#[A-Za-z0-9]+', '', text)
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        tokens = [x for x in tokens if x not in stop_words]
        # Lemmatize the text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(x) for x in tokens]
        # Join the tokens back into a string
        text = ' '.join(tokens)
  return text

In [ ]:
# apply the function to the original 'Text' and create a new column with clean text
df['Text_clean'] = df['Text'].apply(clean_text)
df.head()

,Text,Sentiment,Text_clean
0,!....THE OWNERS REALLY REALLY need to quit bei...,0,owner really really need quit soooooo cheap le...
1,"(It wasn't busy either) Also, the building was...",0,wasnt busy either also building freezing cold
2,(The bathroom is just next door and very nice.),1,bathroom next door nice
3,* Both the Hot & Sour & the Egg Flower Soups w...,1,hot sour egg flower soup absolutely star
4,2 Thumbs Up!!,1,thumb


In [ ]:
# # Import and Instantiate the VADER sentiment Analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment_analyzer = SentimentIntensityAnalyzer()
# Create a function to classify sentiment into positive and Neutral
def get_sentiment_label (score, threshold=0.0):
    if score > threshold:
        return 1
    else:
        return 0
# Split data into train and test sets
x=df['Text_clean']
y=df['Sentiment']
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state= 23)
# Optimize score threshold
best_accuracy = 0.0
best_threshold = 0.0
for threshold in [0.1*i for i in range (-10, 11)]:
  y_pred = x_test.apply(lambda x:
get_sentiment_label(sentiment_analyzer.polarity_scores (x) ['compound'], threshold-threshold))
  accuracy = accuracy_score(y_test, y_pred)
if accuracy > best_accuracy:
  best_accuracy = accuracy
  best_threshold = threshold

In [ ]:
#print the best thresold value
best_threshold
# Apply best threshold to sentiment analysis and create a new variaable in the original data frame
df['sentiment_label_VADER'] = df['Text'].apply(lambda x:
get_sentiment_label (sentiment_analyzer.polarity_scores (x) ['compound'], threshold=best_threshold))

1.0

In [ ]:
# Calculate accuracy

accuracy = accuracy_score (y, df['sentiment_label_VADER'])

print('Accuracy: {:.2f}%'.format(accuracy * 100))

Accuracy: 50.00%


# Naïve Bayes Classifier for Ham and Spam prediction

In [ ]:
#We will use this data- https://archive.ics.uci.edu/ml/datasets/sms+spam+collection
# Save this data in a csv and import in Python
# Please review the 'readme in the downloaded zip folder to get to know more about this data
import numpy as np
import pandas as pd
import nltk
nltk.download ('all')
# import nltk
# nltk.download('punkt')
# import nltk
# nltk.download('stopwords')
# import nltk
# nltk.download('wordnet')
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Magic command to print multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [ ]:
# Read the CSV file, specifying the encoding
df = pd.read_csv('yelp_sentiment (1).csv', encoding = 'unicode_escape')
df.head()
df.shape

,Text,Sentiment
0,!....THE OWNERS REALLY REALLY need to quit bei...,0
1,"(It wasn't busy either) Also, the building was...",0
2,(The bathroom is just next door and very nice.),1
3,* Both the Hot & Sour & the Egg Flower Soups w...,1
4,2 Thumbs Up!!,1


(996, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       996 non-null    object
 1   Sentiment  996 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


In [ ]:
df.rename(columns={'Sentiment': 'Label'}, inplace=True)

In [ ]:
def clean_text(text):
  if True:
        # Convert the text to lowercase
        text = str(text).lower()
        # Remove special characters and digits
        text = re.sub(r'[^a-z\s]', '', text)
        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', '', text)
        # Remove Punctuations
        text = re.sub(r'[^\w\s]', '', text)
        # Remove Numbers
        text = re.sub(r'\d+', '', text)
        # Remove extra white spaces
        text = re.sub(r'\s+', ' ', text).strip()
        # Remove @ mentions
        text = re.sub(r'@\w+', '', text)
        # Remove hashtags
        text = re.sub(r'#[A-Za-z0-9]+', '', text)
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        tokens = [x for x in tokens if x not in stop_words]
        # Lemmatize the text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(x) for x in tokens]
        # Join the tokens back into a string
        text = ' '.join(tokens)
  return text

In [ ]:
#apply the function to the original 'Text' and create a new column with clean text
df['Text_Clean'] = df['Text'].apply(clean_text)

In [ ]:
# Export csv file to visualize in spreadsheet
df.to_csv('cleaned_data.csv')

In [ ]:
# Now let's begin with NB modeling by creatings features (x) and label (y)
x = df.Text_Clean
y = df.Label
x.head()
y.head()
x.shape
print()
y.shape

0    owner really really need quit soooooo cheap le...
1        wasnt busy either also building freezing cold
2                              bathroom next door nice
3             hot sour egg flower soup absolutely star
4                                                thumb
Name: Text_Clean, dtype: object

0    0
1    0
2    1
3    1
4    1
Name: Label, dtype: int64

(996,)

(996,)

In [ ]:
# Vectorize the x by TFIDF vectorizer
vectorizer = TfidfVectorizer()
x_vect = vectorizer.fit_transform(x)
print(x_vect.shape) # of features are 7881 below
print(type(x_vect))
print(vectorizer.get_feature_names_out()[0:10]) # print the first 10 words/features
print (x_vect) # print each feature and the TFIDF score
print(vectorizer.get_feature_names_out() [1077]) # print TFIDF score for word sitting at 7456 position

(996, 1806)
<class 'scipy.sparse._csr.csr_matrix'>
['absolute' 'absolutely' 'absolutley' 'accident' 'accommodation'
 'accomodate' 'accountant' 'ache' 'acknowledged' 'across']
  (0, 1077)	0.1941734012215154
  (0, 1120)	0.2746223683050806
  (0, 1655)	0.24045790320174742
  (0, 1356)	0.23601906791298966
  (0, 614)	0.303824637733646
  (0, 1785)	0.2867424051819794
  (0, 883)	0.2746223683050806
  (0, 247)	0.2867424051819794
  (0, 1463)	0.303824637733646
  (0, 1248)	0.303824637733646
  (0, 1035)	0.24542009887651517
  (0, 1270)	0.36892289312840826
  (0, 1107)	0.24542009887651517
  (1, 284)	0.34989708915807977
  (1, 616)	0.45041850769744546
  (1, 192)	0.425094183273145
  (1, 25)	0.28558966631739735
  (1, 487)	0.3721741034157753
  (1, 201)	0.4071262891482565
  (1, 1727)	0.3288818848665863
  (2, 1044)	0.4062514412290358
  (2, 441)	0.5699008745858146
  (2, 1043)	0.47385051160518527
  (2, 112)	0.5344515434510028
  (3, 1491)	0.3108375717531069
  :	:
  (989, 602)	0.2791445797918721
  (990, 117)	0.7578

In [ ]:
# Split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x_vect, y)
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(747, 1806)

(249, 1806)

(747,)

(249,)

In [ ]:
#Build the NB model and make predictions on teh test data to get classification report
from sklearn.metrics import classification_report as cr
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print (cr(y_test, y_pred))

MultinomialNB()

              precision    recall  f1-score   support

           0       0.80      0.80      0.80       116
           1       0.83      0.83      0.83       133

    accuracy                           0.82       249
   macro avg       0.81      0.81      0.81       249
weighted avg       0.82      0.82      0.82       249



In [ ]:
# Make predictions for the entire data and append into the original data
df['Predicted'] = clf.predict(x_vect)
df.sample(10)

,Text,Label,Text_Clean,Predicted
39,An excellent new restaurant by an experienced ...,1,excellent new restaurant experienced frenchman,1
649,The croutons also taste homemade which is an e...,1,crouton also taste homemade extra plus,1
54,Appetite instantly gone.,0,appetite instantly gone,0
770,The steaks are all well trimmed and also perfe...,1,steak well trimmed also perfectly cooked,1
60,"As much as I'd like to go back, I can't get pa...",0,much id like go back cant get passed atrocious...,0
805,There was hardly any meat.,0,hardly meat,0
461,My fella got the huevos rancheros and they did...,0,fella got huevos rancheros didnt look appealing,0
472,My sashimi was poor quality being soggy and ta...,0,sashimi poor quality soggy tasteless,0
380,I'm not really sure how Joey's was voted best ...,0,im really sure joeys voted best hot dog valley...,0
425,It's worth driving up from Tucson!,1,worth driving tucson,1


# Insights
* Negative Class Performance (Class 0):

Precision: 81%
Recall: 79%
F1 Score: 77%
The classifier effectively identifies negative samples with a high precision score, indicating precise negative sentiment classification. However, it occasionally misses some negative samples, as reflected in the recall rate.

* Positive Class Performance (Class 1):

Precision: 82%
Recall: 80%
F1 Score: 81%
Similar to the negative class, the classifier demonstrates strong precision in recognizing positive samples. Nevertheless, it occasionally misses positive sentiments, as indicated by the recall score.

* Overall Performance Assessment:

Accuracy: 80%
Balanced Precision, Recall, and F1 Scores for Both Classes
When considering the confusion matrix and overall performance metrics, the Naive Bayes sentiment classifier proves its competence on the test dataset. It achieves an 80% accuracy rate and maintains solid precision, recall, and F1 scores for both negative and positive sentiment classes.

* Room for Improvement:

Enhancing Recall Rates
Despite its overall strong performance, there is potential for improvement in recall scores for both negative and positive classes. This enhancement would reduce the number of missed sentiment classifications and further refine the classifier's accuracy.

In summary, the Naive Bayes sentiment classifier provides a reliable sentiment classification model with good accuracy, precision, recall, and F1 scores. However, it can benefit from fine-tuning to enhance recall rates for more comprehensive sentiment recognition.